In [1]:
from tools.data_reading import CompressedWindowDataset
import torch
from torch import nn

In [2]:
dataset = CompressedWindowDataset()
print(dataset[0])
len(dataset[0])

torch.Size([2137, 24, 9, 12])
torch.Size([2263, 3, 96, 128])
2137
(tensor([[[  0.,   0.,   0.,  ...,  65.,  66.,  74.],
         [  0.,   0.,   0.,  ...,  65.,  66.,  74.],
         [  0.,   0.,   0.,  ...,  65.,  66.,  74.],
         ...,
         [ 57.,  58.,  59.,  ..., 132., 127., 124.],
         [ 57.,  57.,  58.,  ..., 132., 128., 126.],
         [ 57.,  57.,  58.,  ..., 131., 127., 126.]],

        [[  0.,   0.,   0.,  ...,  68.,  69.,  77.],
         [  0.,   0.,   0.,  ...,  68.,  69.,  77.],
         [  0.,   0.,   0.,  ...,  68.,  69.,  77.],
         ...,
         [ 68.,  69.,  70.,  ..., 133., 127., 124.],
         [ 68.,  68.,  69.,  ..., 131., 127., 125.],
         [ 68.,  68.,  69.,  ..., 130., 126., 125.]],

        [[  0.,   0.,   0.,  ...,  59.,  60.,  68.],
         [  0.,   0.,   0.,  ...,  59.,  60.,  68.],
         [  0.,   0.,   0.,  ...,  59.,  60.,  68.],
         ...,
         [ 60.,  61.,  62.,  ..., 128., 125., 122.],
         [ 60.,  60.,  61.,  ..., 127.,

3